In [1]:
import pandas as pd
import os
import copy
import numpy as np

### 查看结果的方法

   1. 解注释下方的cross_val变量,和task变量
   2. 然后依次运行下面代码，即可看到最后结果
   3. shift+回车键 快捷执行代码

In [38]:
model = "MLP"   # RF LR MLP KNN可选

#task = "DMAC"
task = "QMAC"

#cross_val = "intra_subject" # 
#cross_val = "LOSO"
cross_val = "mixed"  # task = QMAC 只能选择这个
result_from = "all_result"
#result_from = "pc_result"
base_dir = f'./{result_from}/{model}'
cur_dir = os.path.join(os.path.join(base_dir, task), cross_val)
rs_dir = os.listdir(cur_dir)
rs_dir_copy = copy.deepcopy(rs_dir)
for rid, rd in enumerate(rs_dir_copy):
    if '.' in rd:
        rs_dir.pop(rid)
print(rs_dir)

['2', '3', '1', '4']


In [39]:
print(f"cur path is :{cur_dir}")
PW0_mean = np.zeros((len(rs_dir), 4), dtype=float)
PW0_std = np.zeros((len(rs_dir), 4), dtype=float)
for rsd in rs_dir:
    if "_" in rsd:
        continue
    PW0_mean[int(rsd)-1, 0] = int(rsd)
    PW0_std[int(rsd)-1, 0] = int(rsd)
    
    cur_rs_dir = os.path.join(cur_dir, rsd)
    all_rs = os.listdir(cur_rs_dir)
    if not all_rs:
        continue
    PW_0_mean = []
    PW_0_std = []
    
    for cur_file in all_rs:
        if not str(cur_file).endswith(".csv"):
            continue
        cur_file_dir = os.path.join(cur_rs_dir, cur_file)
        #pw_i = int(str(cur_file).split(".")[0][-1])
        if cross_val != "LOSO":
            # read intra-sub result out
            # 新的顺序 cohen_kappa_s, micro_precision_s, macro_precision_s, weight_precision_s, roc_score, f1, acc
            rs = np.loadtxt(cur_file_dir, dtype=float, delimiter=",", skiprows=0, usecols=[4, 5, 6])
            # rs = pd.read_csv(cur_file_dir, sep=",", names=["AUC_ROC", "Kappa", "Acc", "macro_P", "weight_P", "f1"])
        else:
            # read loso result out
            rs = np.loadtxt(cur_file_dir, dtype=float, delimiter=",", skiprows=1, usecols=[4, 5, 6])
            # rs = pd.read_csv(cur_file_dir, sep=",", names=["AUC_ROC", "Kappa", "Acc", "macro_P", "weight_P", "f1"], header=0)
            
        mean = np.mean(rs, axis=0, keepdims=True)
        std = np.std(rs, axis=0, keepdims=True)
        
        PW_0_mean.append(mean)
        PW_0_std.append(std)
    print("\n")
    print(f"当前窗口为：{rsd}s")    
    print(pd.DataFrame(np.concatenate(PW_0_mean, axis=0), columns=['AUC_ROC', 'F1-score', 'acc'], dtype=float))
        
    PW0_mean[int(rsd)-1, 1:] = np.mean(np.concatenate(PW_0_mean, axis=0), axis=0, keepdims=True)[0]
    PW0_std[int(rsd)-1, 1:] = np.std(np.concatenate(PW_0_std, axis=0), axis=0, keepdims=True)[0] if cross_val != "LOSO" else np.concatenate(PW_0_std, axis=0)

cur path is :./all_result/MLP/QMAC/mixed


当前窗口为：2s
     AUC_ROC  F1-score       acc
0   0.654496  0.652422  0.652422
1   0.642626  0.641362  0.641362
2   0.649782  0.647730  0.647730
3   0.641668  0.638636  0.638636
4   0.652220  0.650078  0.650078
5   0.644052  0.642122  0.642122
6   0.648920  0.647122  0.647122
7   0.650186  0.648410  0.648410
8   0.652802  0.650910  0.650910
9   0.652130  0.649924  0.649924
10  0.645906  0.643714  0.643714
11  0.651126  0.649090  0.649090
12  0.649780  0.647348  0.647348
13  0.646954  0.644316  0.644316
14  0.652694  0.650756  0.650756
15  0.651634  0.649090  0.649090
16  0.639060  0.637728  0.637728
17  0.655572  0.654318  0.654318
18  0.656554  0.654242  0.654242
19  0.647424  0.646286  0.646286


当前窗口为：3s
     AUC_ROC  F1-score       acc
0   0.613472  0.602322  0.602322
1   0.623610  0.617678  0.617678
2   0.602154  0.599464  0.599464
3   0.622902  0.619464  0.619464
4   0.618378  0.614286  0.614286
5   0.626910  0.624286  0.624286
6   0.607878 

In [40]:
rs = [PW0_mean, PW0_std]
rs_name = ["mean", "std"]
for name, content in zip(rs_name, rs):
    print(f"--------------------------{name}-----------------------------")
    c = pd.DataFrame(content, columns=['SW', 'AUC_ROC', 'Acc', 'F1-score'], dtype=float)
    print(c)
    print("\n")

--------------------------mean-----------------------------
    SW   AUC_ROC       Acc  F1-score
0  1.0  0.598509  0.595899  0.595899
1  2.0  0.649279  0.647280  0.647280
2  3.0  0.616512  0.610018  0.610018
3  4.0  0.660186  0.658283  0.658283


--------------------------std-----------------------------
    SW   AUC_ROC       Acc  F1-score
0  1.0  0.002989  0.002905  0.002905
1  2.0  0.005246  0.005476  0.005476
2  3.0  0.006622  0.007294  0.007294
3  4.0  0.005047  0.004846  0.004846




In [41]:
pd.DataFrame(PW0_mean, columns=['SW', 'AUC_ROC', 'Acc', 'F1-score'], dtype=float)

,SW,AUC_ROC,Acc,F1-score
0,1.0,0.598509,0.595899,0.595899
1,2.0,0.649279,0.647280,0.647280
2,3.0,0.616512,0.610018,0.610018
3,4.0,0.660186,0.658283,0.658283


In [42]:
pd.DataFrame(PW0_std, columns=['SW', 'AUC_ROC', 'Acc', 'F1-score'], dtype=float)

,SW,AUC_ROC,Acc,F1-score
0,1.0,0.002989,0.002905,0.002905
1,2.0,0.005246,0.005476,0.005476
2,3.0,0.006622,0.007294,0.007294
3,4.0,0.005047,0.004846,0.004846
